# Visualize Citoid evaluation results

This is an interactive notebook that allows to consult the results of the evaluation of [Citoid](https://www.mediawiki.org/wiki/Citoid) for the [Web2Cit-Research](https://meta.wikimedia.org/wiki/Web2Cit/Research) project.

__Usage:__

First, run the cells to load the data and the functions. Then, interact with the widgets to visualize plots and samples.

__Author:__

* Nidia Hernández, [nidiahernandez@conicet.gov.ar](mailto:nidiahy@gmail.com), CAICYT-CONICET


## Table of Contents

0. Load evaluation results 
1. Show samples of best, middle and low scores
2. Strict vs relaxed scores
3. Filter by language
4. Filter by field
5. Results by domain

In [1]:
import os
import pandas as pd
import gzip
from glob import glob
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## 0. Load evaluation results

We have evaluated 321k citations, comparing the metadata returned by Citoid with the metadata of a manual citation for the following information: source type, title, author (first name(s) and last names(s)), publishig source and publishing date. 

The results of the comparison are in the columns beginning with the prefix "comp_".

In [5]:
results = pd.read_csv("eval/eval_all_citations.csv.gz")

In [3]:
results.shape

(321179, 25)

In [4]:
results.head()

,Unnamed: 0,url,url_citoid,source_type_citoid,title_citoid,author_first_citoid,author_last_citoid,pub_date_citoid,pub_source_citoid,source_type_manual,...,article_url,page_id,revid,wiki_lang,comp_author_first,comp_author_last,comp_title,comp_source_type,comp_pub_source,comp_pub_date
0,0,http://www.bbc.co.uk/programmes/b00rq1w3,https://www.bbc.co.uk/programmes/b00rq1w3,webpage,"bbc radio 4 extra - james bond, goldfinger",NaN,NaN,NaN,bbc,"['blogPost', 'email', 'forumPost', 'webpage']",...,https://en.wikipedia.org/wiki/Goldfinger%20%28...,168425,1047941668,en,1.0,1.00,0.24,1,1.00,1.0
1,1,http://www.gtaforums.com/index.php?showtopic=4...,https://gtaforums.com/topic/393961-title-updat...,webpage,title update v. 1.0.2.0,NaN,NaN,NaN,gtaforums,"['magazineArticle', 'newspaperArticle']",...,https://es.wikipedia.org/wiki/Grand%20Theft%20...,557425,140622011,es,1.0,1.00,0.35,0,0.69,1.0
2,2,https://archive.org/details/incharacter00john,http://archive.org/details/incharacter00john,book,in character,[''],['internet archive'],1984,harmondsworth : penguin,"['book', 'manuscript']",...,https://en.wikipedia.org/wiki/Laurence%20Olivier,43675,1069785118,en,0.0,0.19,1.00,1,0.57,0.7
3,3,http://news.ph.msn.com/business/article.aspx?c...,http://www.msn.com/en-ph/news?cp-documentid=36...,webpage,msn,NaN,NaN,NaN,www.msn.com,"['blogPost', 'email', 'forumPost', 'webpage']",...,https://es.wikipedia.org/wiki/Airbus%20A380,45307,141455347,es,1.0,0.00,0.02,1,0.18,0.0
4,4,http://news.ph.msn.com/business/article.aspx?c...,http://www.msn.com/en-ph/news?cp-documentid=36...,webpage,msn,NaN,NaN,NaN,www.msn.com,[],...,https://pt.wikipedia.org/wiki/Airbus%20A380,77972,62917519,pt,1.0,1.00,0.02,0,0.00,0.0


## 1. Show samples of best, middle and low scores

In this section, you can inspect the data and the evaluation results filtered by fieldname. The first menu allows to select the number of samples to retrieve. The second menu, the field to consult. 

The output shows samples of data and scoring using a relaxed match: best scored cases appear on top (70-100% coincidence), middle scores in the center (40-70% coincidence) and low scores at the bottom (less than 40%). There response from Citoid is displayed at left, the attended response is at the center, and the score is presented at right.

In [3]:
fieldnames = [
    "source_type", "title",
    "author_first", "author_last",
    "pub_source", "pub_date",
]

In [6]:
pd.set_option("colwidth", 200)

def show_samples(n, fieldname):
    discrete_scores = fieldname in ["source_type", "pub_date"]

    best_max = 1 if discrete_scores else 0.9
    
    ## Best scores
    display(results[
        (0.7 < results[f"comp_{fieldname}"]) &
        (results[f"comp_{fieldname}"] <= best_max) ## do not show None vs None cases
    ][[f"{fieldname}_citoid", f"{fieldname}_manual", f"comp_{fieldname}"]].sample(n))

    ## Mostrar el 0.7 de pub date!
    if not discrete_scores:
        ## Middle scores
        display(results[
            (0.4 < results[f"comp_{fieldname}"]) &
            (results[f"comp_{fieldname}"] <= 0.7)
        ][[f"{fieldname}_citoid", f"{fieldname}_manual", f"comp_{fieldname}"]].sample(n))

    ## Low scores
    display(results[
        #(0.1 < results[f"comp_{fieldname}"]) & ## do not show all 0
        (results[f"comp_{fieldname}"] < 0.4)
    ][[f"{fieldname}_citoid", f"{fieldname}_manual", f"comp_{fieldname}"]].sample(n))
    
interact(show_samples, n=[5, 10, 15], fieldname=fieldnames)

interactive(children=(Dropdown(description='n', options=(5, 10, 15), value=5), Dropdown(description='fieldname…

<function __main__.show_samples(n, fieldname)>

## 2. Strict vs window scores

In this section, you can inspect the proportion of each match type for each fieldname. You can select strict or window scoring. The strict scoring considers a fullmatch only for 100% coincidence and no match for 0% coincidence. The window scoring admits full match starting at 70% but considers no match from 40% to lower.

In [18]:
## Strict scoring 
# full_match: 100% match (score == 1)
# no_match: score == 0
# partial_match: the rest

rows = []

total = len(results)

for fieldname in fieldnames:
    full_match = len(results[ results[f"comp_{fieldname}"] == 1 ])
    no_match = len(results[ results[f"comp_{fieldname}"] == 0])
    partial_match = total - full_match - no_match
    
    row = {"full_match":round(full_match/total,2), "partial_match":round(partial_match/total,2), "no_match": round(no_match/total,2)}
    rows.append(row)
    
totals_strict = pd.DataFrame(rows)
totals_strict.index = fieldnames

#totals_strict

In [8]:
totals_strict.mean(axis=0).round(2) * 100

full_match       45.0
partial_match    25.0
no_match         30.0
dtype: float64

In [21]:
## Window scoring
# full_match: score > 0.7
# partial_match:  0.7 >= score > 0.3
# no_match: score > 0.3


fieldnames = [
    "source_type", "title",
    "author_first", "author_last",
    "pub_source", "pub_date",
]

rows = []

total = len(results)

for fieldname in fieldnames:
    #print(fieldname)
    full_match = len(results[ results[f"comp_{fieldname}"] > 0.7 ])
    
    no_match = len(results[ results[f"comp_{fieldname}"] <= 0.3])
    
    partial_match = total - full_match - no_match
    
    row = {"full_match":round(full_match/total,2), "partial_match":round(partial_match/total,2), "no_match": round(no_match/total,2)}
    rows.append(row)
    
totals_window = pd.DataFrame(rows)
totals_window.index = fieldnames

#totals_window

In [22]:
totals_window.mean(axis=0).round(2) * 100

full_match       48.0
partial_match     8.0
no_match         44.0
dtype: float64

In [23]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def plot_stacked_ratios(key):

    colors = ["Green", "Orange", "Black"]

    df = {
        "Strict": totals_strict,
        "Window": totals_window,
    }[key]

    ax = df.plot.bar(stacked=True, color=colors, title=key)
    ax.legend(bbox_to_anchor=(1, 1))
    
    plt.show()
    
    display(df)
    

interact(
    plot_stacked_ratios,
    key=["Strict", "Window"],
)

interactive(children=(Dropdown(description='key', options=('Strict', 'Window'), value='Strict'), Output()), _d…

<function __main__.plot_stacked_ratios(key)>

## 3. Filter by language

In this section, the user can filter the data by the language of the wikipedia and compare the results between fields.

In [24]:
comp_cols = [f"comp_{field}" for field in fieldnames]

data_per_lang = pd.concat([
    (results.set_index("wiki_lang")[comp_cols] == 1).groupby("wiki_lang").mean().assign(match_type="full match"),
    (~results.set_index("wiki_lang")[comp_cols].isin({0, 1})).groupby("wiki_lang").mean().assign(match_type="partial match"),
    (results.set_index("wiki_lang")[comp_cols] == 0).groupby("wiki_lang").mean().assign(match_type="no match")
]).round(2).sort_values(by="wiki_lang").reset_index().set_index(["wiki_lang", "match_type"]).transpose()
data_per_lang.index = fieldnames

data_per_lang

wiki_lang            en                                es                \
match_type   full match partial match no match full match partial match   
source_type        0.66          0.00     0.34       0.78          0.00   
title              0.31          0.67     0.02       0.25          0.72   
author_first       0.50          0.07     0.43       0.57          0.05   
author_last        0.45          0.15     0.40       0.41          0.17   
pub_source         0.22          0.54     0.25       0.18          0.50   
pub_date           0.37          0.14     0.49       0.42          0.10   

wiki_lang                     fr                                pt  \
match_type   no match full match partial match no match full match   
source_type      0.22       0.90          0.00     0.10       0.82   
title            0.03       0.20          0.78     0.02       0.30   
author_first     0.38       0.79          0.01     0.21       0.65   
author_last      0.41       0.55          0.09     0.36       0.46   
pub_source       0.32       0.17          0.58     0.25       0.20   
pub_date         0.47       0.57          0.06     0.37       0.44   

wiki_lang                            
match_type   partial match no match  
source_type           0.00     0.18  
title                 0.66     0.04  
author_first          0.03     0.32  
author_last           0.13     0.41  
pub_source            0.49     0.31  
pub_date              0.07     0.48

In [25]:
colors = {
    "full match": "Green",
    "partial match": "Orange",
    "no match": "Black"
}

def plot_stacked_ratios(lang):
    # data_per_lang esta definido afuera
    data_for_plot = data_per_lang.loc[:, lang]
    ax = data_for_plot.plot.bar(stacked=True, color=colors, title=lang)
    ax.legend(bbox_to_anchor=(1, 1))
    
    plt.show()
    display(data_for_plot)

interact(
    plot_stacked_ratios,
    lang=results["wiki_lang"].unique(),
)

interactive(children=(Dropdown(description='lang', options=('en', 'es', 'pt', 'fr'), value='en'), Output()), _…

<function __main__.plot_stacked_ratios(lang)>

## 4. Filter by field

In this section, the user can filter the data by fieldname and compare the results between different Wikipedias.

In [26]:
def plot_field_by_lang(field):
    df = data_per_lang.loc[field].reset_index().pivot_table(index="wiki_lang", columns="match_type", values=field)
    ax = df.plot.bar(stacked=True, color=colors, title=field)
    ax.legend(bbox_to_anchor=(1, 1))
    
    plt.show()
    display(df)
    
interact(
    plot_field_by_lang,
    field=fieldnames,
)

interactive(children=(Dropdown(description='field', options=('source_type', 'title', 'author_first', 'author_l…

<function __main__.plot_field_by_lang(field)>

# 5. Results by domain